In [2]:
import numpy as np

def cpu_matrix_multiplication(A, B):

  return np.dot(A, B)


A = np.random.rand(1024, 1024).astype(np.float32)
B = np.random.rand(1024, 1024).astype(np.float32)

print("--- CPU Matrix Multiplication ---")
%timeit cpu_matrix_multiplication(A, B)

--- CPU Matrix Multiplication ---
11.1 ms ± 1.88 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
from numba import cuda
import numpy as np
import math

@cuda.jit
def gpu_matrix_multiplication_kernel(A, B, C):
  """
  CUDA kernel for matrix multiplication.
  """
  row, col = cuda.grid(2)
  if row < C.shape[0] and col < C.shape[1]:
    tmp = 0.
    for k in range(A.shape[1]):
      tmp += A[row, k] * B[k, col]
    C[row, col] = tmp

def gpu_matrix_multiplication(A, B):
  """
  Sets up and runs the CUDA kernel for matrix multiplication.
  """
  # Allocate memory on the device
  A_device = cuda.to_device(A)
  B_device = cuda.to_device(B)
  C_device = cuda.device_array_like(A)

  # Configure the blocks and grids
  threadsperblock = (32, 32)
  blockspergrid_x = math.ceil(A.shape[0] / threadsperblock[0])
  blockspergrid_y = math.ceil(B.shape[1] / threadsperblock[1])
  blockspergrid = (blockspergrid_x, blockspergrid_y)

  # Launch the kernel
  gpu_matrix_multiplication_kernel[blockspergrid, threadsperblock](A_device, B_device, C_device)

  # Copy the result back to the host
  return C_device.copy_to_host()


# --- Benchmarking ---
# Create two large matrices
A = np.random.rand(1024, 1024).astype(np.float32)
B = np.random.rand(1024, 1024).astype(np.float32)

print("\n--- GPU Matrix Multiplication ---")
%timeit gpu_matrix_multiplication(A, B)



--- GPU Matrix Multiplication ---
40.4 ms ± 2.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
import numpy as np

def cpu_convolution(signal, kernel):
  """
  Performs 1D convolution on the CPU using NumPy.
  """
  return np.convolve(signal, kernel, mode='same')

# --- Benchmarking ---
signal = np.random.rand(1000000).astype(np.float32)
kernel = np.random.rand(256).astype(np.float32)

print("--- CPU 1D Convolution ---")
%timeit cpu_convolution(signal, kernel)

--- CPU 1D Convolution ---
23.8 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
from numba import cuda
import numpy as np
import math

@cuda.jit
def gpu_convolution_kernel(signal, kernel, output):
  """
  CUDA kernel for 1D convolution.
  """
  idx = cuda.grid(1)
  kernel_radius = kernel.shape[0] // 2
  if idx < output.shape[0]:
    temp = 0.
    for i in range(kernel.shape[0]):
      k_idx = i - kernel_radius
      s_idx = idx + k_idx
      if s_idx >= 0 and s_idx < signal.shape[0]:
        temp += signal[s_idx] * kernel[i]
    output[idx] = temp

def gpu_convolution(signal, kernel):
  """
  Sets up and runs the CUDA kernel for 1D convolution.
  """
  output = np.zeros_like(signal)
  signal_device = cuda.to_device(signal)
  kernel_device = cuda.to_device(kernel)
  output_device = cuda.to_device(output)

  threadsperblock = 256
  blockspergrid = math.ceil(signal.shape[0] / threadsperblock)

  gpu_convolution_kernel[blockspergrid, threadsperblock](signal_device, kernel_device, output_device)

  return output_device.copy_to_host()

# --- Benchmarking ---
signal = np.random.rand(1000000).astype(np.float32)
kernel = np.random.rand(256).astype(np.float32)

print("\n--- GPU 1D Convolution ---")
%timeit gpu_convolution(signal, kernel)


--- GPU 1D Convolution ---
12.2 ms ± 557 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
import numpy as np

def cpu_moving_average(data, window_size):
  """
  Calculates the moving average on the CPU using NumPy.
  """
  return np.convolve(data, np.ones(window_size), 'valid') / window_size

# --- Benchmarking ---
data = np.random.rand(5000000).astype(np.float32)
window_size = 100

print("--- CPU Moving Average ---")
%timeit cpu_moving_average(data, window_size)

--- CPU Moving Average ---
129 ms ± 5.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
from numba import cuda
import numpy as np
import math

@cuda.jit
def gpu_moving_average_kernel(data, window_size, output):
  """
  CUDA kernel for calculating the moving average.
  """
  idx = cuda.grid(1)
  if idx < output.shape[0]:
    temp = 0.
    for i in range(window_size):
      temp += data[idx + i]
    output[idx] = temp / window_size

def gpu_moving_average(data, window_size):
  """
  Sets up and runs the CUDA kernel for moving average.
  """
  output = np.zeros(data.shape[0] - window_size + 1, dtype=np.float32)
  data_device = cuda.to_device(data)
  output_device = cuda.to_device(output)

  threadsperblock = 256
  blockspergrid = math.ceil(output.shape[0] / threadsperblock)

  gpu_moving_average_kernel[blockspergrid, threadsperblock](data_device, window_size, output_device)

  return output_device.copy_to_host()

# --- Benchmarking ---
data = np.random.rand(5000000).astype(np.float32)
window_size = 100

print("\n--- GPU Moving Average ---")
%timeit gpu_moving_average(data, window_size)


--- GPU Moving Average ---
31 ms ± 1.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
import numpy as np

def cpu_correlation(signal1, signal2):
  """
  Performs 1D cross-correlation on the CPU using NumPy.
  'full' mode provides the complete cross-correlation.
  """
  return np.correlate(signal1, signal2, mode='full')

# --- Benchmarking ---
signal1 = np.random.randn(5000).astype(np.float32)
signal2 = np.random.randn(5000).astype(np.float32)

print("--- CPU Correlation ---")
%timeit cpu_correlation(signal1, signal2)

--- CPU Correlation ---
1.49 ms ± 130 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [3]:
from numba import cuda
import numpy as np
import math

@cuda.jit
def gpu_correlation_kernel(signal1, signal2, output):
  """
  CUDA kernel for 1D cross-correlation.
  """
  lag = cuda.grid(1)
  
  if lag < output.shape[0]:
    # The starting point in the output corresponds to a specific overlap
    n_signal1 = signal1.shape[0]
    n_signal2 = signal2.shape[0]
    
    start_index = lag - (n_signal2 - 1)
    
    # Calculate the sum of products for the current lag
    corr_sum = 0.0
    for i in range(n_signal1):
      j = i - start_index
      if 0 <= j < n_signal2:
        corr_sum += signal1[i] * signal2[j]
    output[lag] = corr_sum

def gpu_correlation(signal1, signal2):
  """
  Sets up and runs the CUDA kernel for 1D correlation.
  """
  output_size = signal1.shape[0] + signal2.shape[0] - 1
  output = np.zeros(output_size, dtype=np.float32)
  
  d_signal1 = cuda.to_device(signal1)
  d_signal2 = cuda.to_device(signal2)
  d_output = cuda.to_device(output)

  threadsperblock = 256
  blockspergrid = math.ceil(output_size / threadsperblock)

  gpu_correlation_kernel[blockspergrid, threadsperblock](d_signal1, d_signal2, d_output)

  return d_output.copy_to_host()

# --- Benchmarking ---
signal1 = np.random.randn(5000).astype(np.float32)
signal2 = np.random.randn(5000).astype(np.float32)

print("\n--- GPU Correlation ---")
%timeit gpu_correlation(signal1, signal2)


--- GPU Correlation ---


c:\Users\Ssn\anaconda3\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 40 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


2.89 ms ± 688 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
from scipy import integrate
import numpy as np

# A sample complex function to integrate
def my_func(x):
  return np.sin(x**2) * np.exp(-x/2)

# --- Benchmarking ---
# integrate.quad is not easily benchmarked with %timeit for a single run,
# but a typical execution is very fast for simple functions.
# We will compare a simple trapezoidal rule on the CPU for a fairer comparison
# against the GPU's trapezoidal rule.

def cpu_trapezoid_integration(func, a, b, n=10_000_000):
    x = np.linspace(a, b, n, dtype=np.float32)
    y = func(x)
    return np.trapz(y, x)

print("--- CPU Numerical Integration (Trapezoid Rule) ---")
%timeit cpu_trapezoid_integration(my_func, 0, 10)

In [ ]:
from numba import cuda
import numpy as np
import math

@cuda.jit
def gpu_trapezoid_kernel(func, a, b, n, partial_sums):
    """
    CUDA kernel to compute the area of many small trapezoids in parallel.
    """
    thread_id = cuda.grid(1)
    threads_total = cuda.gridsize(1)

    # Each thread computes its share of the trapezoids
    dx = (b - a) / n
    local_sum = 0.0
    
    for i in range(thread_id, n, threads_total):
        x1 = a + i * dx
        x2 = a + (i + 1) * dx
        
        # Area of one trapezoid
        local_sum += (func(x1) + func(x2)) * dx / 2.0
        
    partial_sums[thread_id] = local_sum

def gpu_integration(func, a, b, n=10_000_000):
  """
  Sets up and runs the CUDA kernel for trapezoidal integration.
  """
  # We need to compile the function for the device
  device_func = cuda.jit(func, device=True)

  threadsperblock = 256
  blockspergrid = 32 # Use a fixed number of blocks for this example
  
  partial_sums = cuda.device_array(threadsperblock * blockspergrid, dtype=np.float32)

  gpu_trapezoid_kernel[blockspergrid, threadsperblock](device_func, 
                                                       np.float32(a), 
                                                       np.float32(b), 
                                                       n, 
                                                       partial_sums)

  # Final sum is done on the host for simplicity, but can also be done on device
  return partial_sums.copy_to_host().sum()

# --- Benchmarking ---
print("\n--- GPU Numerical Integration ---")
%timeit gpu_integration(my_func, 0, 10)